In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
ORG_NAME = "Decentraland"
SPLITS_FREQ = "W-THU"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2023-07-13"


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2213 entries, 0 to 2212
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2213 non-null   object        
 1   author             2213 non-null   object        
 2   date               2213 non-null   datetime64[us]
 3   start              2213 non-null   datetime64[us]
 4   end                2213 non-null   datetime64[us]
 5   platform_proposal  2213 non-null   object        
dtypes: datetime64[us](3), object(3)
memory usage: 103.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        116560 non-null  object        
 1   proposal  116560 non-null  object        
 2   voter     116560 non-null  object        
 3   date      116560 non-null  datetime64[

In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x30b1f4bd5476906f38385b891f2c09973196b742,312c5654-0806-5473-8243-dbd6311208ed,2021-05-24 15:29:29,1
1,0x30b1f4bd5476906f38385b891f2c09973196b742,f6bd4c7d-0d1e-5437-9cf9-e5e883522d3f,2021-05-24 15:30:54,1
2,0xd689478d44a438798ee0dc07657cce2135c0aef7,2dc28092-bb92-5c3f-8837-f8ed570d1a20,2021-05-24 15:38:54,1
3,0x862f109696d7121438642a78b3caa38f476db08b,2dc28092-bb92-5c3f-8837-f8ed570d1a20,2021-05-24 15:41:31,1
4,0x862f109696d7121438642a78b3caa38f476db08b,94c28bf3-501e-5925-98a3-b4ddbed07abf,2021-05-24 15:50:36,1
...,...,...,...,...
116555,0x28bf51f47e903925c00a03264c7e7a0576785600,e5fcfaed-fd89-547f-aba0-b682b2114a4b,2023-07-19 16:19:59,1
116556,0xdcd35e70c43c6f86c663a1919259cd9e818d9865,f41ed318-2f75-5a16-a798-6623e99fb55a,2023-07-19 17:54:31,1
116557,0x598f8af1565003ae7456dac280a18ee826df7a2c,858339c3-6902-52c5-ab98-bb5985327cbd,2023-07-19 17:57:35,1
116558,0x059d0025e4e1bb1a37f2c922e54b139bc1231ee0,f41ed318-2f75-5a16-a798-6623e99fb55a,2023-07-20 00:56:09,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2023-05-11 00:00:00'),
 Timestamp('2023-05-18 00:00:00'),
 Timestamp('2023-05-25 00:00:00'),
 Timestamp('2023-06-01 00:00:00'),
 Timestamp('2023-06-08 00:00:00'),
 Timestamp('2023-06-15 00:00:00'),
 Timestamp('2023-06-22 00:00:00'),
 Timestamp('2023-06-29 00:00:00'),
 Timestamp('2023-07-06 00:00:00'),
 Timestamp('2023-07-13 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2023-07-13 00:00:00
time_train                    5.586113
time_rec                      0.235893
open_proposals                      23
min_recs                             1
avg_recs                     17.139785
precision@1                   0.311828
precision@3                   0.247312
precision@5                   0.219355
precision@10                  0.194624
precision@15                  0.172043
precision@100                 0.028495
ndcg@1                         0.18239
ndcg@3                        0.209188
ndcg@5                        0.231742
ndcg@10                        0.28868
ndcg@15                       0.323575
ndcg@100                      0.343855
map@1                         0.311828
map@3                         0.315114
map@5                         0.334677
map@10                         0.38935
map@15                        0.421812
map@100                       0.436076
recall@1                      0.168364
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/Decentraland/hparams-itemknn_W-THU_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-05-11,1,7d,2023-05-11,0.001979,0.078880,18,3,12.038961,0.181818,...,0.763714,0.939703,1.0,0.496514,0.462125,0.517725,0.573589,0.573589,0.573589,0.150749
1,2023-05-11,1,14d,2023-05-11,0.002496,0.104145,18,1,9.795918,0.408163,...,0.710398,0.931365,1.0,0.462738,0.725047,0.759164,0.797215,0.797215,0.797215,0.151467
2,2023-05-11,1,21d,2023-05-11,0.003510,0.117980,18,1,10.269231,0.394231,...,0.703709,0.930517,1.0,0.488037,0.735422,0.766104,0.802771,0.802771,0.802771,0.153098
3,2023-05-11,1,30d,2023-05-11,0.006359,0.146640,18,1,11.068966,0.353448,...,0.666831,0.885981,1.0,0.488037,0.735422,0.766104,0.802771,0.802771,0.802771,0.157500
4,2023-05-11,1,60d,2023-05-11,0.016443,0.197784,18,1,11.669291,0.322835,...,0.653431,0.886999,1.0,0.488037,0.724118,0.738547,0.779782,0.779782,0.779782,0.160813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2023-07-13,15,21d,2023-07-13,0.003096,0.099991,23,1,17.322917,0.322917,...,0.670474,0.850437,1.0,0.548104,0.566449,0.619459,0.671726,0.698084,0.698084,0.167174
766,2023-07-13,15,30d,2023-07-13,0.007112,0.110362,23,1,17.656863,0.333333,...,0.679576,0.854547,1.0,0.553663,0.572137,0.614578,0.668901,0.694665,0.694665,0.169142
767,2023-07-13,15,60d,2023-07-13,0.009500,0.131727,23,1,18.045455,0.272727,...,0.661726,0.844898,1.0,0.577346,0.552368,0.599579,0.653939,0.682712,0.683962,0.172257
768,2023-07-13,15,90d,2023-07-13,0.022476,0.163614,23,1,18.532787,0.245902,...,0.640354,0.827367,1.0,0.560679,0.529617,0.569309,0.627930,0.655074,0.656253,0.176710


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train   avg_recs  precision@5  precision@10  \
window_size k                                                      
14d         1     0.002657  11.795176     0.294811      0.238330   
            2     0.003055  11.795176     0.292367      0.238533   
            5     0.003758  11.795176     0.289380      0.238968   
            6     0.002990  11.795176     0.288702      0.239057   
            8     0.003988  11.795176     0.287542      0.239146   
...                    ...        ...          ...           ...   
10YE        9     1.856116  13.553344     0.219389      0.200243   
            6     1.859945  13.553344     0.220312      0.199587   
            10    1.856279  13.553344     0.219760      0.200427   
            8     1.858787  13.553344     0.219414      0.200321   
            7     1.860542  13.553344     0.219232      0.199880   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
14d         1        0.032617  0.251338  0.305121  0.355726  0.370794   
            2        0.032617  0.249115  0.304077  0.354753  0.368169   
            5        0.032617  0.247244  0.304047  0.354617  0.365416   
            6        0.032617  0.246669  0.303816  0.354269  0.364055   
            8        0.032617  0.246146  0.303855  0.354189  0.363455   
...                       ...       ...       ...       ...       ...   
10YE        9        0.029795  0.282115  0.375205  0.466980  0.273332   
            6        0.029795  0.282835  0.374438  0.467069  0.273919   
            10       0.029795  0.281827  0.374894  0.466456  0.272597   
            8        0.029795  0.281869  0.374915  0.466390  0.272628   
            7        0.029795  0.281227  0.373800  0.465981  0.272037   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
14d         1   0.424955  0.467311  0.469874   0.727755         1.0   
            2   0.423584  0.465934  0.464267   0.725686         1.0   
            5   0.423329  0.465307  0.456290   0.725791         1.0   
            6   0.422576  0.464448  0.455529   0.726606         1.0   
            8   0.422444  0.464297  0.454255   0.727183         1.0   
...                  ...       ...       ...        ...         ...   
10YE        9   0.333141  0.380670  0.358219   0.655553         1.0   
            6   0.332779  0.380674  0.359606   0.652113         1.0   
            10  0.332531  0.379938  0.359135   0.656419         1.0   
            8   0.332356  0.379846  0.359539   0.656525         1.0   
            7   0.331611  0.379326  0.359264   0.654190         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
14d         1        0.640825        0.702385         0.725938  
            2        0.646940        0.707584         0.730491  
            5        0.645491        0.709220         0.731628  
            6        0.642165        0.705489         0.727921  
            8        0.639623        0.703894         0.726460  
...                       ...             ...              ...  
10YE        9        0.611211        0.672199         0.697477  
            6        0.611674        0.673913         0.699634  
            10       0.610576        0.671041         0.696141  
            8        0.610373        0.671142         0.696496  
            7        0.609936        0.672263         0.697896  

[77 rows x 17 columns]

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Decentraland/models/itemknn-best-avg_W-THU_normalize.parquet


/tmp/ipykernel_3791646/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-05-11,14d,1,2023-05-11,0.002496,0.104145,18,1,9.795918,0.408163,0.360544,0.285714,0.197959,...,0.710398,0.931365,1.0,0.462738,0.725047,0.759164,0.797215,0.797215,0.797215,0.151467
2023-05-18,14d,1,2023-05-18,0.002507,0.128965,25,1,17.720000,0.256000,0.304000,0.321600,0.291200,...,0.437241,0.649508,1.0,0.217968,0.367656,0.500451,0.619004,0.691548,0.712603,0.183733
2023-05-25,14d,1,2023-05-25,0.003606,0.099112,19,1,11.462366,0.397849,0.315412,0.264516,0.203226,...,0.670176,0.817996,1.0,0.490296,0.619946,0.678793,0.740604,0.756181,0.758632,0.153816
2023-06-01,14d,1,2023-06-01,0.002102,0.081702,13,1,8.246753,0.350649,0.376623,0.368831,0.296104,...,0.864974,1.000000,1.0,0.339169,0.553594,0.657643,0.711178,0.715219,0.715219,0.145793
2023-06-08,14d,1,2023-06-08,0.001708,0.078551,12,1,7.090909,0.389610,0.372294,0.322078,0.253247,...,0.982843,1.000000,1.0,0.433295,0.628332,0.710119,0.760888,0.763105,0.763105,0.143809
2023-06-15,14d,1,2023-06-15,0.002537,0.079439,16,1,10.884615,0.346154,0.371795,0.333333,0.267949,...,0.739228,0.979167,1.0,0.375619,0.608941,0.698010,0.751046,0.761147,0.761147,0.153030
2023-06-22,14d,1,2023-06-22,0.001895,0.104415,17,1,10.388350,0.252427,0.239482,0.227184,0.173786,...,0.617599,0.902515,1.0,0.555355,0.610403,0.694314,0.732228,0.739221,0.739221,0.155831
2023-06-29,14d,1,2023-06-29,0.003029,0.062535,10,1,6.666667,0.350000,0.344444,0.330000,0.231667,...,1.000000,1.000000,1.0,0.453175,0.638690,0.748851,0.754598,0.754598,0.754598,0.141177
2023-07-06,14d,1,2023-07-06,0.002182,0.134483,21,1,16.557143,0.285714,0.273810,0.272857,0.247143,...,0.578065,0.782473,1.0,0.254965,0.382057,0.479769,0.595455,0.627566,0.639302,0.190520


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Decentraland/models/itemknn-best-val_W-THU_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-05-11,1,14d,0.002496,9.795918,0.285714,0.197959,0.026122,0.356626,0.391901,0.454883,0.466905,0.498433,0.539179,0.568732,0.710398,1.0,0.759164,0.797215,0.797215
2023-05-18,1,14d,0.002507,17.720000,0.321600,0.291200,0.056320,0.229720,0.269276,0.393060,0.289909,0.324614,0.427065,0.262887,0.437241,1.0,0.500451,0.619004,0.712603
2023-05-25,1,14d,0.003606,11.462366,0.264516,0.203226,0.028925,0.324086,0.368930,0.437620,0.409277,0.437601,0.480852,0.456975,0.670176,1.0,0.678793,0.740604,0.758632
2023-06-01,1,7d,0.005079,9.508197,0.409836,0.321311,0.035082,0.306202,0.377056,0.395295,0.455624,0.553738,0.574304,0.591828,0.905514,1.0,0.612826,0.654334,0.658562
2023-06-08,8,7d,0.004669,7.507463,0.340299,0.256716,0.025970,0.322154,0.385177,0.386287,0.469469,0.543141,0.544497,0.692593,0.993668,1.0,0.610780,0.649704,0.652035
2023-06-15,1,14d,0.002537,10.884615,0.333333,0.267949,0.034615,0.231754,0.274912,0.318069,0.425260,0.473385,0.521752,0.496843,0.739228,1.0,0.698010,0.751046,0.761147
2023-06-22,1,14d,0.001895,10.388350,0.227184,0.173786,0.025049,0.202069,0.239697,0.301537,0.323603,0.364652,0.410909,0.437645,0.617599,1.0,0.694314,0.732228,0.739221
2023-06-29,1,7d,0.001495,7.266667,0.400000,0.224444,0.022444,0.218270,0.235395,0.235395,0.621049,0.648630,0.648630,0.845556,1.000000,1.0,0.854487,0.854487,0.854487
2023-07-06,4,7d,0.001859,16.650794,0.284127,0.262698,0.040079,0.164918,0.202432,0.253117,0.326299,0.374346,0.438123,0.348822,0.620192,1.0,0.503200,0.624296,0.668284


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-06-15 00:00:00,0.003198,0.088100,17.333333,2.000000,12.058693,0.366613,0.347663,0.310661,0.244353,...,0.743300,0.901306,1.0,0.415658,0.560207,0.641369,0.699714,0.718688,0.722562,0.158764
min,2023-05-18 00:00:00,0.001495,0.045035,10.000000,1.000000,7.266667,0.252427,0.239482,0.215054,0.173786,...,0.437241,0.649508,1.0,0.217968,0.367656,0.500451,0.619004,0.652035,0.652035,0.139926
25%,2023-06-01 00:00:00,0.001895,0.064580,13.000000,1.000000,9.508197,0.311828,0.304000,0.264516,0.203226,...,0.620192,0.878588,1.0,0.314394,0.523362,0.610780,0.649704,0.658562,0.668284,0.143835
50%,2023-06-15 00:00:00,0.002537,0.094833,17.000000,1.000000,10.884615,0.365079,0.315412,0.321600,0.256716,...,0.706080,0.902515,1.0,0.415724,0.576632,0.619459,0.671726,0.698084,0.712603,0.153816
75%,2023-06-29 00:00:00,0.004669,0.104415,21.000000,3.000000,16.650794,0.426230,0.383085,0.340299,0.267949,...,0.905514,1.000000,1.0,0.504348,0.610403,0.694314,0.740604,0.756181,0.758632,0.165495
max,2023-07-13 00:00:00,0.005134,0.128965,25.000000,5.000000,17.720000,0.511111,0.474074,0.409836,0.321311,...,1.000000,1.000000,1.0,0.567513,0.767949,0.854487,0.854487,0.854487,0.854487,0.189645
std,NaN,0.001454,0.028059,5.123475,1.414214,4.087506,0.085507,0.083869,0.069166,0.047950,...,0.189166,0.115350,0.0,0.122676,0.115204,0.108403,0.076538,0.066243,0.064407,0.017668


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Decentraland/models/itemknn-best-test_W-THU_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-05-18,1,14d,0.002507,17.720000,0.321600,0.291200,0.056320,0.229720,0.269276,0.393060,0.289909,0.324614,0.427065,0.262887,0.437241,1.0,0.500451,0.619004,0.712603
2023-05-25,1,14d,0.003606,11.462366,0.264516,0.203226,0.028925,0.324086,0.368930,0.437620,0.409277,0.437601,0.480852,0.456975,0.670176,1.0,0.678793,0.740604,0.758632
2023-06-01,1,14d,0.002102,8.246753,0.368831,0.296104,0.032597,0.332933,0.429548,0.460311,0.406144,0.503720,0.524596,0.523947,0.864974,1.0,0.657643,0.711178,0.715219
2023-06-08,1,7d,0.001483,7.507463,0.343284,0.256716,0.025970,0.313573,0.374251,0.375360,0.438209,0.512681,0.514038,0.705031,0.993668,1.0,0.567219,0.608247,0.610704
2023-06-15,8,7d,0.001590,11.229508,0.324590,0.290164,0.032787,0.172996,0.224218,0.241155,0.363005,0.455314,0.479442,0.520492,0.857728,1.0,0.538889,0.598677,0.609788
2023-06-22,1,14d,0.001895,10.388350,0.227184,0.173786,0.025049,0.202069,0.239697,0.301537,0.323603,0.364652,0.410909,0.437645,0.617599,1.0,0.694314,0.732228,0.739221
2023-06-29,1,14d,0.003029,6.666667,0.330000,0.231667,0.023167,0.221274,0.274687,0.274687,0.448181,0.525654,0.525654,0.670278,1.000000,1.0,0.748851,0.754598,0.754598
2023-07-06,1,7d,0.001807,16.650794,0.298413,0.267460,0.040079,0.161754,0.196660,0.244945,0.312941,0.357103,0.415569,0.366250,0.632310,1.0,0.482119,0.614293,0.659854
2023-07-13,4,7d,0.001924,18.662162,0.189189,0.172973,0.027973,0.159360,0.200181,0.255807,0.282980,0.328336,0.382633,0.382633,0.613625,1.0,0.572339,0.622133,0.642291


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-06-15 00:00:00,0.002216,0.087942,17.333333,2.222222,12.059340,0.330038,0.301894,0.296401,0.242588,...,0.743036,0.893815,1.0,0.392893,0.521953,0.604513,0.666774,0.685205,0.689212,0.157755
min,2023-05-18 00:00:00,0.001483,0.060505,10.000000,1.000000,6.666667,0.252427,0.211712,0.189189,0.172973,...,0.437241,0.649508,1.0,0.217968,0.367656,0.482119,0.598677,0.609788,0.609788,0.141177
25%,2023-06-01 00:00:00,0.001807,0.064464,13.000000,1.000000,8.246753,0.283784,0.291005,0.264516,0.203226,...,0.617599,0.819981,1.0,0.326568,0.467602,0.538889,0.614293,0.642291,0.642291,0.145793
50%,2023-06-15 00:00:00,0.001924,0.081702,17.000000,1.000000,11.229508,0.344262,0.306011,0.321600,0.256716,...,0.670176,0.902515,1.0,0.390690,0.553594,0.572339,0.622133,0.691548,0.712603,0.153816
75%,2023-06-29 00:00:00,0.002507,0.104415,21.000000,3.000000,16.650794,0.350649,0.328358,0.330000,0.290164,...,0.864974,1.000000,1.0,0.490296,0.610403,0.678793,0.732228,0.739221,0.739221,0.161426
max,2023-07-13 00:00:00,0.003606,0.128965,25.000000,5.000000,18.662162,0.417910,0.376623,0.368831,0.296104,...,1.000000,1.000000,1.0,0.555355,0.638690,0.748851,0.754598,0.756181,0.758632,0.185747
std,NaN,0.000705,0.025446,5.123475,1.563472,4.540035,0.058254,0.050555,0.058426,0.049444,...,0.193788,0.119050,0.0,0.119711,0.101255,0.093394,0.065681,0.059402,0.059534,0.016628


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2023-07-13 00:00:00
time_train                    0.600163
time_rec                       0.91582
open_proposals                      23
min_recs                             1
avg_recs                     17.139785
precision@1                   0.290323
precision@3                   0.265233
precision@5                    0.24086
precision@10                  0.197849
precision@15                   0.17276
precision@100                 0.028495
ndcg@1                        0.169811
ndcg@3                        0.201805
ndcg@5                        0.231522
ndcg@10                       0.282153
ndcg@15                       0.316005
ndcg@100                      0.336224
map@1                         0.290323
map@3                         0.294504
map@5                          0.31612
map@10                        0.369978
map@15                        0.401226
map@100                       0.414376
recall@1                      0.123561
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/Decentraland/hparams-userknn_W-THU_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2023-05-11,1,7d,2023-05-11,0.000935,0.059704,18,3,12.038961,0.181818,...,0.803108,0.954314,1.0,0.257823,0.447844,0.492669,0.534148,0.534148,0.534148,0.153198
1,2023-05-11,1,14d,2023-05-11,0.001486,0.170670,18,1,9.795918,0.397959,...,0.758734,0.939869,1.0,0.472039,0.732073,0.768452,0.814462,0.814462,0.814462,0.152285
2,2023-05-11,1,21d,2023-05-11,0.002138,0.077512,18,1,10.269231,0.375000,...,0.771211,0.936126,1.0,0.474603,0.739381,0.791071,0.830750,0.830750,0.830750,0.152791
3,2023-05-11,1,30d,2023-05-11,0.003730,0.090430,18,1,11.068966,0.344828,...,0.732348,0.891728,1.0,0.488333,0.742460,0.777673,0.812176,0.812176,0.812176,0.156191
4,2023-05-11,1,60d,2023-05-11,0.007980,0.110077,18,1,11.669291,0.346457,...,0.727250,0.897235,1.0,0.497348,0.733503,0.740749,0.782279,0.782279,0.782279,0.159652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2023-07-13,15,21d,2023-07-13,0.002061,0.077100,23,1,17.322917,0.208333,...,0.706519,0.872040,1.0,0.429322,0.468440,0.548060,0.595927,0.625840,0.625840,0.169437
766,2023-07-13,15,30d,2023-07-13,0.002583,0.083701,23,1,17.656863,0.245098,...,0.709160,0.898548,1.0,0.465201,0.478298,0.561482,0.607267,0.634152,0.634152,0.170935
767,2023-07-13,15,60d,2023-07-13,0.004600,0.092636,23,1,18.045455,0.227273,...,0.685615,0.883482,1.0,0.459291,0.468394,0.526560,0.587143,0.619511,0.620931,0.171413
768,2023-07-13,15,90d,2023-07-13,0.008154,0.108871,23,1,18.532787,0.262295,...,0.699512,0.873442,1.0,0.400288,0.440017,0.494752,0.567326,0.594275,0.595605,0.177379


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train   avg_recs  precision@5  precision@10  \
window_size k                                                      
7d          5     0.000753  12.647853     0.304664      0.249541   
            6     0.000750  12.647853     0.303619      0.249592   
14d         1     0.001192  11.795176     0.304061      0.245481   
7d          7     0.000746  12.647853     0.304733      0.250321   
            4     0.000745  12.647853     0.305521      0.249511   
...                    ...        ...          ...           ...   
10YE        7     0.063071  13.553344     0.237056      0.206213   
            8     0.063306  13.553344     0.234777      0.205882   
            9     0.063011  13.553344     0.234360      0.205493   
            10    0.063161  13.553344     0.232407      0.205459   
            15    0.063335  13.553344     0.231263      0.203562   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          5        0.032067  0.219782  0.270015  0.302404  0.385177   
            6        0.032067  0.218293  0.269037  0.301706  0.383414   
14d         1        0.032617  0.266991  0.322971  0.366753  0.388714   
7d          7        0.032067  0.217695  0.268817  0.300891  0.381038   
            4        0.032067  0.218433  0.268639  0.300915  0.380697   
...                       ...       ...       ...       ...       ...   
10YE        7        0.029795  0.315340  0.401940  0.485677  0.304772   
            8        0.029795  0.313201  0.400446  0.484884  0.303057   
            9        0.029795  0.313450  0.399939  0.484988  0.302787   
            10       0.029795  0.308933  0.397328  0.482403  0.298318   
            15       0.029795  0.306435  0.392669  0.480353  0.295961   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          5   0.450708  0.487322  0.527108   0.797472         1.0   
            6   0.449708  0.486307  0.523787   0.795562         1.0   
14d         1   0.448234  0.487013  0.498387   0.766443         1.0   
7d          7   0.447549  0.483443  0.524117   0.799515         1.0   
            4   0.446100  0.482697  0.527094   0.797792         1.0   
...                  ...       ...       ...        ...         ...   
10YE        7   0.361351  0.405982  0.412348   0.687839         1.0   
            8   0.360155  0.404940  0.408900   0.684645         1.0   
            9   0.359767  0.404753  0.408857   0.682271         1.0   
            10  0.356374  0.401432  0.402889   0.682014         1.0   
            15  0.352836  0.398582  0.398703   0.670966         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          5        0.559802        0.614051         0.631457  
            6        0.566049        0.622607         0.640789  
14d         1        0.624739        0.688862         0.710271  
7d          7        0.556721        0.613929         0.632194  
            4        0.556510        0.610442         0.628508  
...                       ...             ...              ...  
10YE        7        0.617533        0.680208         0.704519  
            8        0.615819        0.679696         0.704394  
            9        0.610750        0.673314         0.697463  
            10       0.613046        0.675728         0.700155  
            15       0.606337        0.667996         0.692577  

[77 rows x 17 columns]

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Decentraland/models/userknn-best-avg_W-THU_normalize.parquet


/tmp/ipykernel_3791646/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2023-05-11,7d,5,2023-05-11,0.000841,0.054410,18,3,12.038961,0.337662,0.298701,0.262338,0.207792,...,0.796614,0.967301,1.0,0.384341,0.485775,0.520819,0.561702,0.561702,0.561702,0.150575
2023-05-18,7d,5,2023-05-18,0.000812,0.075316,25,4,18.150943,0.292453,0.301887,0.284906,0.297170,...,0.499045,0.727278,1.0,0.155899,0.274841,0.335670,0.458817,0.512494,0.523958,0.182835
2023-05-25,7d,5,2023-05-25,0.000873,0.052974,19,4,13.120000,0.360000,0.297778,0.264000,0.208000,...,0.803101,0.955444,1.0,0.487316,0.523123,0.561717,0.608647,0.616165,0.616165,0.151440
2023-06-01,7d,5,2023-06-01,0.000588,0.042611,13,5,9.508197,0.377049,0.420765,0.350820,0.322951,...,0.921908,1.000000,1.0,0.323518,0.505947,0.531602,0.576263,0.580592,0.580592,0.143193
2023-06-08,7d,5,2023-06-08,0.000578,0.046905,12,3,7.507463,0.402985,0.378109,0.331343,0.253731,...,0.984547,1.000000,1.0,0.412567,0.529658,0.592594,0.642631,0.645088,0.645088,0.144073
2023-06-15,7d,5,2023-06-15,0.000663,0.042816,16,4,11.229508,0.524590,0.409836,0.370492,0.286885,...,0.866315,1.000000,1.0,0.379489,0.514835,0.566434,0.612989,0.621119,0.621119,0.148157
2023-06-22,7d,5,2023-06-22,0.000672,0.057863,17,1,11.734940,0.361446,0.265060,0.233735,0.186747,...,0.694399,0.946386,1.0,0.435659,0.505194,0.583365,0.608457,0.614226,0.614226,0.153188
2023-06-29,7d,5,2023-06-29,0.000635,0.032679,10,2,7.266667,0.377778,0.407407,0.391111,0.224444,...,1.000000,1.000000,1.0,0.638235,0.667262,0.790476,0.790476,0.790476,0.790476,0.139024
2023-07-06,7d,5,2023-07-06,0.000993,0.089445,21,3,16.650794,0.309524,0.285714,0.304762,0.275397,...,0.710753,0.892814,1.0,0.343438,0.422569,0.515778,0.625088,0.656877,0.668062,0.190454


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Decentraland/models/userknn-best-val_W-THU_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2023-05-11,1,14d,0.001486,9.795918,0.308163,0.210204,0.026122,0.374433,0.409749,0.461353,0.486749,0.520029,0.553336,0.622413,0.758734,1.0,0.768452,0.814462,0.814462
2023-05-18,1,14d,0.001351,17.720000,0.337600,0.304800,0.056320,0.256184,0.298079,0.407881,0.320940,0.353318,0.446648,0.300316,0.502784,1.0,0.462033,0.587228,0.672671
2023-05-25,2,7d,0.000855,13.120000,0.269333,0.210667,0.026000,0.287832,0.337823,0.369861,0.421974,0.476151,0.505723,0.579365,0.819545,1.0,0.639739,0.687982,0.694785
2023-06-01,10,7d,0.000595,9.508197,0.380328,0.321311,0.035082,0.291082,0.372120,0.390412,0.432354,0.545973,0.566241,0.568604,0.905514,1.0,0.574066,0.619450,0.623679
2023-06-08,2,7d,0.000571,7.507463,0.322388,0.253731,0.025970,0.312245,0.381132,0.384246,0.462537,0.546120,0.550190,0.669293,0.984547,1.0,0.638403,0.687123,0.689515
2023-06-15,8,7d,0.000662,11.229508,0.383607,0.290164,0.032787,0.221877,0.255094,0.271261,0.495264,0.555505,0.579895,0.633997,0.868657,1.0,0.585595,0.639663,0.647996
2023-06-22,1,21d,0.001333,10.864865,0.252252,0.184685,0.025405,0.248649,0.290382,0.344397,0.367182,0.406180,0.445776,0.479688,0.687314,1.0,0.692720,0.738114,0.751774
2023-06-29,15,7d,0.000635,7.266667,0.408889,0.224444,0.022444,0.216408,0.230539,0.230539,0.606235,0.629874,0.629874,0.874074,1.000000,1.0,0.805556,0.805556,0.805556
2023-07-06,6,7d,0.000930,16.650794,0.304762,0.278571,0.040079,0.173887,0.216423,0.254711,0.325683,0.386933,0.442242,0.438445,0.725567,1.0,0.523715,0.630947,0.673922


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000
mean,0.000996,12.354490,0.322082,0.251463,0.032503,0.253238,0.300166,0.336475,0.422683,0.480067,0.514839,0.559560,0.800436,1.0,0.613696,0.671514,0.692192
min,0.000571,7.266667,0.239583,0.184685,0.022444,0.173887,0.216423,0.230539,0.320940,0.353318,0.442242,0.300316,0.502784,1.0,0.462033,0.587228,0.623679
25%,0.000635,9.508197,0.269333,0.210667,0.025970,0.221877,0.255094,0.271261,0.367182,0.406180,0.446648,0.479688,0.709992,1.0,0.574066,0.630947,0.669831
50%,0.000855,11.229508,0.322388,0.253731,0.028438,0.256184,0.298079,0.369861,0.421974,0.476151,0.505723,0.568604,0.819545,1.0,0.601437,0.647566,0.673922
75%,0.001333,16.650794,0.380328,0.290164,0.035082,0.287832,0.337823,0.384246,0.462537,0.546120,0.566241,0.633997,0.905514,1.0,0.639739,0.687982,0.694785
max,0.002030,17.720000,0.408889,0.321311,0.056320,0.312245,0.381132,0.407881,0.606235,0.629874,0.629874,0.874074,1.000000,1.0,0.805556,0.805556,0.805556
std,0.000489,4.084785,0.060950,0.050033,0.010514,0.043433,0.058665,0.066247,0.090866,0.093939,0.069303,0.161926,0.160063,0.0,0.098712,0.067050,0.055088


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Decentraland/models/userknn-best-test_W-THU_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2023-05-18,1,14d,2023-05-18,0.001351,0.089021,25,1,17.720000,0.312000,0.338667,0.337600,0.304800,...,0.502784,0.712118,1.0,0.185168,0.347887,0.462033,0.587228,0.652367,0.672671,0.263365
2023-05-25,1,14d,2023-05-25,0.001394,0.067287,19,1,11.462366,0.397849,0.301075,0.273118,0.213978,...,0.733110,0.877396,1.0,0.512786,0.621569,0.694706,0.760442,0.776052,0.778555,0.152984
2023-06-01,2,7d,2023-06-01,0.000586,0.042415,13,5,9.508197,0.295082,0.377049,0.331148,0.322951,...,0.920815,1.000000,1.0,0.343013,0.463674,0.505808,0.556818,0.561147,0.561147,0.142843
2023-06-08,10,7d,2023-06-08,0.000568,0.046724,12,3,7.507463,0.388060,0.328358,0.325373,0.253731,...,0.984547,1.000000,1.0,0.370743,0.492577,0.560742,0.616813,0.619410,0.619410,0.142318
2023-06-15,2,7d,2023-06-15,0.000683,0.042482,16,4,11.229508,0.459016,0.415301,0.344262,0.283607,...,0.861176,1.000000,1.0,0.308702,0.449332,0.504861,0.557440,0.563690,0.563690,0.148690
2023-06-22,8,7d,2023-06-22,0.000709,0.059397,17,1,11.734940,0.373494,0.244980,0.231325,0.191566,...,0.726527,0.946386,1.0,0.399384,0.483349,0.565417,0.591152,0.597070,0.597070,0.157059
2023-06-29,1,21d,2023-06-29,0.001396,0.051041,10,1,7.058824,0.338235,0.318627,0.314706,0.225000,...,1.000000,1.000000,1.0,0.511594,0.670000,0.761979,0.767187,0.767187,0.767187,0.142368
2023-07-06,15,7d,2023-07-06,0.000954,0.089877,21,3,16.650794,0.269841,0.275132,0.287302,0.278571,...,0.705792,0.914734,1.0,0.319211,0.429260,0.510218,0.623824,0.658307,0.670441,0.188958
2023-07-13,6,7d,2023-07-13,0.000966,0.054288,23,5,18.662162,0.297297,0.243243,0.216216,0.190541,...,0.691778,0.885972,1.0,0.478713,0.517033,0.553539,0.605144,0.626077,0.626077,0.160195


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2023-06-15 00:00:00,0.000956,0.060281,17.333333,2.666667,12.392695,0.347875,0.315826,0.295672,0.251638,...,0.791837,0.926290,1.0,0.381035,0.497187,0.568812,0.629561,0.646812,0.650694,0.166531
min,2023-05-18 00:00:00,0.000568,0.042415,10.000000,1.000000,7.058824,0.269841,0.243243,0.216216,0.190541,...,0.502784,0.712118,1.0,0.185168,0.347887,0.462033,0.556818,0.561147,0.561147,0.142318
25%,2023-06-01 00:00:00,0.000683,0.046724,13.000000,1.000000,9.508197,0.297297,0.275132,0.273118,0.213978,...,0.705792,0.885972,1.0,0.319211,0.449332,0.505808,0.587228,0.597070,0.597070,0.142843
50%,2023-06-15 00:00:00,0.000954,0.054288,17.000000,3.000000,11.462366,0.338235,0.318627,0.314706,0.253731,...,0.733110,0.946386,1.0,0.370743,0.483349,0.553539,0.605144,0.626077,0.626077,0.152984
75%,2023-06-29 00:00:00,0.001351,0.067287,21.000000,4.000000,16.650794,0.388060,0.338667,0.331148,0.283607,...,0.920815,1.000000,1.0,0.478713,0.517033,0.565417,0.623824,0.658307,0.672671,0.160195
max,2023-07-13 00:00:00,0.001396,0.089877,25.000000,5.000000,18.662162,0.459016,0.415301,0.344262,0.322951,...,1.000000,1.000000,1.0,0.512786,0.670000,0.761979,0.767187,0.776052,0.778555,0.263365
std,NaN,0.000347,0.018347,5.123475,1.732051,4.317189,0.061171,0.057545,0.046977,0.048904,...,0.162247,0.094971,0.0,0.107965,0.097634,0.097703,0.079548,0.078469,0.079719,0.039120
